**Name: Sakshi Ubale**

**Title: Fraud Detection**

# **Load data using kaggle API**

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
! kaggle datasets download mlg-ulb/creditcardfraud

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
 89% 59.0M/66.0M [00:01<00:00, 65.2MB/s]
100% 66.0M/66.0M [00:01<00:00, 63.7MB/s]


In [3]:
!unzip creditcardfraud.zip

Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [4]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# ***Data Loading***

In [5]:
df=pd.read_csv('creditcard.csv')

In [6]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [7]:
df.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


# ***Feature Extraction***

In [8]:
x=df.drop('Class',axis=1)
y=df['Class']

In [9]:
x.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [10]:
#The label where 0 represents a normal transaction and 1 represents a fraudulent one.
y.head()

,Class
0,0
1,0
2,0
3,0
4,0


# ***Scaling the extracted features***

In [11]:
sc=StandardScaler()
ss_x=sc.fit_transform(x)

In [12]:
ss_x

array([[-1.99658302, -0.69424232, -0.04407492, ...,  0.33089162,
        -0.06378115,  0.24496426],
       [-1.99658302,  0.60849633,  0.16117592, ..., -0.02225568,
         0.04460752, -0.34247454],
       [-1.99656197, -0.69350046, -0.81157783, ..., -0.13713686,
        -0.18102083,  1.16068593],
       ...,
       [ 1.6419735 ,  0.98002374, -0.18243372, ...,  0.01103672,
        -0.0804672 , -0.0818393 ],
       [ 1.6419735 , -0.12275539,  0.32125034, ...,  0.26960398,
         0.31668678, -0.31324853],
       [ 1.64205773, -0.27233093, -0.11489898, ..., -0.00598394,
         0.04134999,  0.51435531]])

# ***PCA***

In [13]:
pca=PCA(n_components=0.95)
x_pca=pca.fit_transform(ss_x)

In [14]:
x_pca

array([[ 0.41663045, -2.53266428,  0.61038419, ..., -0.4983405 ,
         0.64443702,  0.7445959 ],
       [-0.39345348, -2.0773255 ,  0.60051457, ...,  0.06401662,
        -0.0370807 , -0.33361581],
       [ 1.86852545, -2.53349699, -0.05519717, ..., -0.95904448,
         0.68730412, -0.02369997],
       ...,
       [-0.19406338,  2.07078798, -0.04106663, ..., -0.38946783,
        -0.15125376,  0.64975722],
       [-0.46608925,  1.22539363,  0.02354462, ..., -0.68061162,
         0.68310794,  3.30804094],
       [ 0.76759651,  1.90590292, -0.16559909, ...,  0.12522879,
        -0.02884679, -0.38451215]])

# ***Train test split***

In [15]:
xtrain,xtest,ytrain,ytest = train_test_split(x_pca,y,test_size=0.3,random_state=0)

In [16]:
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape

((199364, 27), (199364,), (85443, 27), (85443,))

# ***Logistic Regression***

In [17]:
lr=LogisticRegression()

In [18]:
lr.fit(xtrain,ytrain)

LogisticRegression()

In [19]:
ypred_lr=lr.predict(xtest)

In [20]:
ypred_lr

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
ytest

,Class
183484,0
255448,0
244749,0
63919,0
11475,0
...,...
208059,0
260930,0
75125,0
37292,0


In [22]:
acc_lr=accuracy_score(ytest,ypred_lr)
acc_lr

0.999204147794436

# ***Decision Tree Classifier***

In [23]:
dc= DecisionTreeClassifier()

In [ ]:
dc.fit(xtrain,ytrain)

In [ ]:
ypred_dc=dc.predict(xtest)

In [ ]:
ypred_dc

In [ ]:
ytest

In [ ]:
acc_dc=accuracy_score(ytest,ypred_dc)
acc_dc

In [ ]:
df['Hour'] = np.mod(df['Time'], 86400) // 3600  # Derive 'hour of the day'

In [ ]:
df.head(10)

# ***Real-time Monitoring system***

In [ ]:
def simulate_top_five_predictions(model, xtest):
    if not isinstance(xtest, pd.DataFrame):
        xtest = pd.DataFrame(xtest)

    for i, transaction in enumerate(xtest.iterrows()):
        if i >= 10:
            break

        transaction_data = transaction[1].values.reshape(1, -1)


        prediction = model.predict(transaction_data)[0]


        if prediction == 1:
            print(f"Transaction {i} is fraudulent! Taking action...")
        else:
            print(f"Transaction {i} is normal.")


        time.sleep(0.5)

simulate_top_five_predictions(dc, xtest)

In [ ]:
df.head(10)

In [ ]:
con_matrix= confusion_matrix(ytest,ypred_dc)
con_matrix

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(con_matrix, annot=True, cmap='viridis', xticklabels=['Normal', 'Fraudulent'], yticklabels=['Normal', 'Fraudulent'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()